In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
data_dir = '/content/drive/MyDrive/ICU_Deterioration_MLOps_Dataset_2012'

folders = os.listdir(data_dir)
folders[:]

['Outcomes-a.txt',
 'Outcomes-b.txt',
 'Outcomes-c.txt',
 'set-b',
 'set-a',
 'set-c']

In [4]:
files = os.listdir(os.path.join(data_dir, 'set-c'))

files[:3]

['158062.txt', '158484.txt', '158364.txt']

In [5]:
file_path = os.path.join(data_dir, 'set-c', files[0])

df = pd.read_csv(file_path)
df.head()

,Time,Parameter,Value
0,00:00,RecordID,158062.0
1,00:00,Age,73.0
2,00:00,Gender,1.0
3,00:00,Height,172.7
4,00:00,ICUType,4.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Time       450 non-null    object 
 1   Parameter  449 non-null    object 
 2   Value      450 non-null    float64
dtypes: float64(1), object(2)
memory usage: 10.7+ KB


In [7]:
def process_patient(file_path):
  record_id = os.path.basename(file_path).split('.')[0]

  df = pd.read_csv(file_path)

  def time_to_hours(time_str):
    h, m = map(int, time_str.split(':'))
    return h + m/60

  df['Time'] = df['Time'].apply(time_to_hours)
  df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
  df.sort_values('Time', inplace=True)
  wide_df = df.pivot_table(
      index='Time',
      columns='Parameter',
      values='Value',
      aggfunc='last'
  )
  wide_df = wide_df.drop(columns=['RecordID'], errors='ignore')
  wide_df = wide_df.sort_index()
  return record_id, wide_df

In [8]:
all_patients_a = []
folder_a = 'set-a'
folder_a_path = os.path.join(data_dir, folder_a)
patient_files_in_folder_a = os.listdir(folder_a_path)
for file_name in patient_files_in_folder_a:
  file_path = os.path.join(folder_a_path, file_name)
  rid, wide_df = process_patient(file_path)
  all_patients_a.append((rid, wide_df))

all_patients_b = []
folder_b = 'set-b'
folder_b_path = os.path.join(data_dir, folder_b)
patient_files_in_folder_b = os.listdir(folder_b_path)
for file_name in patient_files_in_folder_b:
  file_path = os.path.join(folder_b_path, file_name)
  rid, wide_df = process_patient(file_path)
  all_patients_b.append((rid, wide_df))

all_patients_c = []
folder_c = 'set-c'
folder_c_path = os.path.join(data_dir, folder_c)
patient_files_in_folder_c = os.listdir(folder_c_path)
for file_name in patient_files_in_folder_c:
  file_path = os.path.join(folder_c_path, file_name)
  rid, wide_df = process_patient(file_path)
  all_patients_c.append((rid, wide_df))

In [9]:
print(len(all_patients_a))
print(len(all_patients_b))
print(len(all_patients_c))

4000
4000
3183


In [10]:
all_patients_a[:1]

[('135265',
  Parameter   Age  Albumin  BUN  Creatinine  DiasABP   GCS  Gender  Glucose  \
  Time                                                                        
  0.000000   34.0      NaN  NaN         NaN      NaN   NaN     0.0      NaN   
  0.150000    NaN      NaN  NaN         NaN     76.0   NaN     NaN      NaN   
  0.983333    NaN      NaN  NaN         NaN     52.0  15.0     NaN      NaN   
  1.983333    NaN      NaN  NaN         NaN     55.0  15.0     NaN      NaN   
  2.983333    NaN      NaN  NaN         NaN     51.0  15.0     NaN      NaN   
  4.233333    NaN      NaN  7.0         0.5      NaN   NaN     NaN    133.0   
  15.483333   NaN      NaN  8.0         0.5      NaN   NaN     NaN    129.0   
  25.816667   NaN      3.5  NaN         NaN      NaN   NaN     NaN      NaN   
  26.983333   NaN      NaN  NaN         NaN     58.0  15.0     NaN      NaN   
  27.983333   NaN      NaN  NaN         NaN     60.0   NaN     NaN      NaN   
  28.983333   NaN      NaN  NaN         

In [11]:
all_patients_b[0][1].head(7)

Parameter,Age,BUN,Creatinine,DiasABP,FiO2,GCS,Gender,Glucose,HCO3,HCT,...,PaCO2,PaO2,Platelets,SaO2,SysABP,Temp,Urine,WBC,Weight,pH
Time,,,,,,,,,,,,,,,,,,,,,
0.000000,47.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.2,NaN
1.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41.0,284.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.37
2.533333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,47.0,308.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.31
2.883333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,46.0,268.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.30
3.950000,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.450000,NaN,NaN,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,122.0,35.9,90.0,NaN,NaN,NaN
4.466667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.24


In [12]:
# Get union of all columns across patients in set a
all_columns_a = set()

for record_id, wide_df in all_patients_a:
    all_columns_a.update(wide_df.columns)

global_columns_a = sorted(list(all_columns_a))
print(len(global_columns_a))
print(global_columns_a)



all_columns_b = set()

for record_id, wide_df in all_patients_b:
    all_columns_b.update(wide_df.columns)

global_columns_b = sorted(list(all_columns_b))
print(len(global_columns_b))
print(global_columns_b)



all_columns_c = set()

for record_id, wide_df in all_patients_c:
    all_columns_c.update(wide_df.columns)

global_columns_c = sorted(list(all_columns_c))
print(len(global_columns_c))

41
['ALP', 'ALT', 'AST', 'Age', 'Albumin', 'BUN', 'Bilirubin', 'Cholesterol', 'Creatinine', 'DiasABP', 'FiO2', 'GCS', 'Gender', 'Glucose', 'HCO3', 'HCT', 'HR', 'Height', 'ICUType', 'K', 'Lactate', 'MAP', 'MechVent', 'Mg', 'NIDiasABP', 'NIMAP', 'NISysABP', 'Na', 'PaCO2', 'PaO2', 'Platelets', 'RespRate', 'SaO2', 'SysABP', 'Temp', 'TroponinI', 'TroponinT', 'Urine', 'WBC', 'Weight', 'pH']
41
['ALP', 'ALT', 'AST', 'Age', 'Albumin', 'BUN', 'Bilirubin', 'Cholesterol', 'Creatinine', 'DiasABP', 'FiO2', 'GCS', 'Gender', 'Glucose', 'HCO3', 'HCT', 'HR', 'Height', 'ICUType', 'K', 'Lactate', 'MAP', 'MechVent', 'Mg', 'NIDiasABP', 'NIMAP', 'NISysABP', 'Na', 'PaCO2', 'PaO2', 'Platelets', 'RespRate', 'SaO2', 'SysABP', 'Temp', 'TroponinI', 'TroponinT', 'Urine', 'WBC', 'Weight', 'pH']
41


In [13]:
aligned_patients_a = []

for record_id, wide_df in all_patients_a:
    df_aligned = wide_df.reindex(columns=global_columns_a)
    aligned_patients_a.append((record_id, df_aligned))

aligned_patients_b = []

for record_id, wide_df in all_patients_b:
    df_aligned = wide_df.reindex(columns=global_columns_b)
    aligned_patients_b.append((record_id, df_aligned))


aligned_patients_c = []

for record_id, wide_df in all_patients_c:
    df_aligned = wide_df.reindex(columns=global_columns_c)
    aligned_patients_c.append((record_id, df_aligned))

In [14]:
aligned_patients_a[0][1].head(4)

Parameter,ALP,ALT,AST,Age,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,DiasABP,...,RespRate,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,pH
Time,,,,,,,,,,,,,,,,,,,,,
0.000000,NaN,NaN,NaN,34.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.5,NaN
0.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,...,16.0,NaN,123.0,37.4,NaN,NaN,NaN,NaN,NaN,NaN
0.983333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,...,13.0,NaN,101.0,NaN,NaN,NaN,100.0,NaN,NaN,NaN
1.983333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,...,14.0,NaN,106.0,NaN,NaN,NaN,140.0,NaN,NaN,NaN


## Now is the time to scale the numeric value..

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_concat = pd.concat([df for rid, df in aligned_patients_a])
scaler.fit(train_concat.values)

scaled_patients_a = []
for rid, df in aligned_patients_a:
    scaled_df = pd.DataFrame(scaler.transform(df.values), columns=df.columns, index=df.index)
    scaled_patients_a.append((rid, scaled_df))

scaled_patients_b = []
for rid, df in aligned_patients_b:
    scaled_df = pd.DataFrame(scaler.transform(df.values), columns=df.columns, index=df.index)
    scaled_patients_b.append((rid, scaled_df))

scaled_patients_c = []
for rid, df in aligned_patients_c:
    scaled_df = pd.DataFrame(scaler.transform(df.values), columns=df.columns, index=df.index)
    scaled_patients_c.append((rid, scaled_df))

In [16]:
scaled_patients_b[0][1].shape

(101, 41)

In [17]:
def get_shape(lst):
    if isinstance(lst, list):
        return (len(lst),) + get_shape(lst[0]) if lst else (0,)
    else:
        return ()

print(get_shape(scaled_patients_a))


(4000,)


## Now, we need to do one thing: Handle missing value and make X-train, X-val..for that we use max length because not all patients have the same number of timestamp/rows

In [18]:
outcomes_a = pd.read_csv(os.path.join(data_dir,'Outcomes-a.txt'))
labels_a = dict(zip(outcomes_a['RecordID'], outcomes_a['In-hospital_death']))

outcomes_b = pd.read_csv(os.path.join(data_dir,'Outcomes-b.txt'))
labels_b = dict(zip(outcomes_b['RecordID'], outcomes_b['In-hospital_death']))

outcomes_c = pd.read_csv(os.path.join(data_dir,'Outcomes-c.txt'))
labels_c = dict(zip(outcomes_c['RecordID'], outcomes_c['In-hospital_death']))


def df_list_to_padded_array(df_list):
    # df_list elements are (record_id, wide_df) tuples
    # Extract only the DataFrames for calculating max_len and num_features
    dataframes_only = [item[1] for item in df_list] #basically wide_df

    max_len = max(df.shape[0] for df in dataframes_only)
    num_features = dataframes_only[0].shape[1]
    array = np.zeros((len(df_list), max_len, num_features), dtype=np.float32)
    record_ids = []

    for i, (record_id, df_patient) in enumerate(df_list):
        record_ids.append(int(record_id))

        df_filled = df_patient.ffill().bfill().fillna(0.0)  # forward/backward fill then remaining 0
        seq_len = df_filled.shape[0]
        array[i, :seq_len, :] = df_filled.values
    return array, record_ids

X_train, record_ids_train = df_list_to_padded_array(scaled_patients_a)
y_train = np.array([labels_a[rid] for rid in record_ids_train])

X_val, record_ids_val = df_list_to_padded_array(scaled_patients_b)
y_val = np.array([labels_b[rid] for rid in record_ids_val])

X_test, record_ids_test = df_list_to_padded_array(scaled_patients_c)
y_test = np.array([labels_c[rid] for rid in record_ids_test])

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
print("y_test shape:", y_test.shape)


X_train shape: (4000, 203, 41)
X_val shape: (4000, 186, 41)
X_test shape: (3183, 215, 41)
y_train shape: (4000,)
y_val shape: (4000,)
y_test shape: (3183,)


In [19]:
y_train[:5]

array([0, 0, 0, 0, 0])

In [20]:
X_train[:2]

array([[[ 0.        ,  0.        ,  0.        , ...,  0.40949258,
         -0.31486338,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.40949258,
         -0.31486338,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.40949258,
         -0.31486338,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.63284636, -0.31792095, -0.31882557, ..., -0.27087846,
         -1.1333007 ,  0.        ],
        [-0.63284636, -0.31792095, -0.31882557, ..., -0.27087846,
         -1.1333007 ,  0.        ],
        [-0.63284636, -0.31792095, -0.31882557, ..., -0.27087846,
         -1.1333007 ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [ ]:
def create_sliding_windows(X, y, window_size=12, horizon=6):
    """
    X: np.array of shape (num_patients, seq_len, num_features)
    y: np.array of shape (num_patients,)
    window_size: number of past timesteps to use as input
    horizon: how many timesteps ahead to predict
    """
    num_patients = X.shape[0]

    X_windows = []
    y_windows = []

    for i in range(num_patients):
      seq_len = X[i].shape[0]

      for start in range(seq_len - horizon):
        end = start + window_size
        if end > seq_len:
          break
        X_windows.append(X[i][start:end])
        y_windows.append(y[i])

    X_windows = np.array(X_windows, dtype=np.float32)
    y_windows = np.array(y_windows, dtype=np.float32)

    return X_windows, y_windows

X_train_windows, y_train_windows = create_sliding_windows(X_train, y_train)
X_val_windows, y_val_windows = create_sliding_windows(X_val, y_val)
X_test_windows, y_test_windows = create_sliding_windows(X_test, y_test)

print("X_train_windows shape:", X_train_windows.shape)
print("y_train_windows shape:", y_train_windows.shape)

print("X_val_windows shape:", X_val_windows.shape)
print("y_val_windows shape:", y_val_windows.shape)

print("X_test_windows shape:", X_test_windows.shape)
print("y_test_windows shape:", y_test_windows.shape)


X_train_windows shape: (768000, 12, 41)
y_train_windows shape: (768000,)
X_val_windows shape: (700000, 12, 41)
y_val_windows shape: (700000,)
X_test_windows shape: (649332, 12, 41)
y_test_windows shape: (649332,)


## Now time for the Model design --  I will be using pytorch for this


In [22]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device Used:", device)

X_train_tensor = torch.tensor(X_train_windows, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_windows, dtype=torch.float32).unsqueeze(1) #(N,1) format
X_val_tensor = torch.tensor(X_val_windows, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_windows, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test_windows, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_windows, dtype=torch.float32).unsqueeze(1)

#creating dataloader
batch_size = 64
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Device Used: cuda


In [23]:
class CNN_GRU_Model(nn.Module):
  def __init__(self, num_features=41, window_size=12):
    super(CNN_GRU_Model, self).__init__()
    self.cnn = nn.Sequential(
        nn.Conv1d(in_channels=num_features, out_channels=32, kernel_size=3, padding=1),
        nn.BatchNorm1d(32),
        nn.ReLU(),
        nn.Dropout(0.2),

        nn.Conv1d(in_channels=32, out_channels=16, kernel_size=3, padding=1),
        nn.BatchNorm1d(16),
        nn.ReLU(),
        nn.Dropout(0.2)
    )
    self.gru = nn.GRU(input_size=16, hidden_size=32, batch_first=True, dropout=0.3)
    self.fc = nn.Sequential(
        nn.Linear(32,16),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(16,1)
    )

  def forward(self, x):
         #our x: (batch, seq_len, num_features)
         # but cnn expects (batch, channels, seq_len) so we swap 1 & 2 index
      x = x.permute(0,2,1)
      x = self.cnn(x)
         # Back to (batch, seq_len, channels) for GRU
      x = x.permute(0,2,1)
      _, h_n = self.gru(x)
      h_n = h_n.squeeze(0)
      output = self.fc(h_n)

      return output

model = CNN_GRU_Model(num_features=X_train_windows.shape[2], window_size=X_train_windows.shape[1]).to(device)
#print(model)

num_pos = (y_train_tensor == 1).sum().item()
num_neg = (y_train_tensor == 0).sum().item()

print("Number of positive samples in training set:", num_pos)
print("Number of negative samples in training set:", num_neg)

pos_weight_val = num_neg / num_pos #which roughly gave around 6
pos_weight= torch.tensor(pos_weight_val, dtype=torch.float32).to(device)
#pos_weight= torch.tensor(3.0).to(device)


loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-4)

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


Number of positive samples in training set: 106368
Number of negative samples in training set: 661632


Training time

In [29]:
from sklearn.metrics import (roc_auc_score, precision_score, recall_score, f1_score, average_precision_score)

num_epochs = 10
for epoch in range(num_epochs):
  model.train()
  train_loss = 0.0
  train_correct = 0
  train_total = 0

  for x_batch, y_batch in train_loader:
    x_batch, y_batch = x_batch.to(device), y_batch.to(device).float()
    optimizer.zero_grad()

    logits = model(x_batch)
    loss = loss_fn(logits, y_batch)

    loss.backward()
    optimizer.step()

    train_loss += loss.item() * x_batch.size(0)

    probs = torch.sigmoid(logits)
    predicted = (probs >= 0.5).float()
    train_correct += (predicted == y_batch).sum().item()
    train_total += y_batch.size(0)

  train_acc = train_correct/train_total
  train_loss /= train_total

  #validation
  model.eval()
  val_loss = 0.0
  val_correct = 0
  val_total = 0

  all_probs = []
  all_labels = []

  with torch.no_grad():
    for x_batch, y_batch in val_loader:
      x_batch, y_batch = x_batch.to(device), y_batch.to(device).float()

      logits = model(x_batch)
      loss = loss_fn(logits, y_batch)

      val_loss += loss.item() * x_batch.size(0)

      probs = torch.sigmoid(logits)

      all_probs.append(probs.cpu())
      all_labels.append(y_batch.cpu())

  all_probs = torch.cat(all_probs).numpy()
  all_labels = torch.cat(all_labels).numpy()

  val_loss /= len(all_labels)

  thresholds = np.linspace(0.05, 0.95, 91)

  target_recall = 0.75
  best_precision = 0
  best_t = 0.5

  for t in thresholds:
      preds = (all_probs >= t).astype(int)

      recall = recall_score(all_labels, preds)
      if recall >= target_recall:
          precision = precision_score(all_labels, preds)
          if precision > best_precision:
              best_precision = precision
              best_t = t


  final_preds = (all_probs >= best_t).astype(int)

  val_prauc = average_precision_score(all_labels, all_probs)
  val_auc = roc_auc_score(all_labels, all_probs)
  val_precision = precision_score(all_labels, final_preds)
  val_recall = recall_score(all_labels, final_preds)
  val_f1 = f1_score(all_labels, final_preds)

  val_acc = (final_preds == all_labels).mean()

  print(f"Epoch [{epoch+1}/{num_epochs}], Train Acc: {train_acc:.3f},Train Loss: {train_loss:.3f}, \
        Val Acc: {val_acc:.3f}, Val Loss: {val_loss:.4f}, Best Threshold: {best_t:.2f}, \
        PR-AUC: {val_prauc:.3f},   AUROC: {val_auc:.3f}, Precision: {val_precision:.3f},\
        Recall: {val_recall:.3f}, F1Score: {val_f1:.3f} \
   ")

Epoch [1/10], Train Acc: 0.862,Train Loss: 0.893,         Val Acc: 0.370, Val Loss: 1.5564, Best Threshold: 0.14,         PR-AUC: 0.246,   AUROC: 0.642, Precision: 0.169,        Recall: 0.876, F1Score: 0.283    
Epoch [2/10], Train Acc: 0.865,Train Loss: 0.891,         Val Acc: 0.360, Val Loss: 1.5468, Best Threshold: 0.07,         PR-AUC: 0.227,   AUROC: 0.623, Precision: 0.168,        Recall: 0.890, F1Score: 0.283    
Epoch [3/10], Train Acc: 0.864,Train Loss: 0.891,         Val Acc: 0.363, Val Loss: 1.5259, Best Threshold: 0.13,         PR-AUC: 0.231,   AUROC: 0.630, Precision: 0.167,        Recall: 0.872, F1Score: 0.280    
Epoch [4/10], Train Acc: 0.864,Train Loss: 0.889,         Val Acc: 0.359, Val Loss: 1.5139, Best Threshold: 0.12,         PR-AUC: 0.233,   AUROC: 0.636, Precision: 0.168,        Recall: 0.887, F1Score: 0.282    
Epoch [5/10], Train Acc: 0.868,Train Loss: 0.888,         Val Acc: 0.370, Val Loss: 1.5671, Best Threshold: 0.19,         PR-AUC: 0.234,   AUROC: 0.629,

In [25]:
# This is result with window size 24:

# Epoch [1/3], Train Acc: 0.8443,Train Loss: 0.9260,         Val Acc: 0.3667, Val Loss: 1.4490, Best Threshold: 0.16         AUROC: 0.6385, Precision: 0.1685,        Recall: 0.8793, F1Score: 0.2828
# Epoch [2/3], Train Acc: 0.8517,Train Loss: 0.9168,         Val Acc: 0.3606, Val Loss: 1.4792, Best Threshold: 0.16         AUROC: 0.6374, Precision: 0.1670,        Recall: 0.8787, F1Score: 0.2807
# Epoch [3/3], Train Acc: 0.8529,Train Loss: 0.9099,         Val Acc: 0.3735, Val Loss: 1.4358, Best Threshold: 0.34         AUROC: 0.6423, Precision: 0.1672,        Recall: 0.8569, F1Score: 0.2798


In [32]:
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0

all_probs = []
all_labels = []

with torch.no_grad():
  for x_batch, y_batch in test_loader:
    x_batch, y_batch = x_batch.to(device), y_batch.to(device).float()

    logits = model(x_batch)
    probs = torch.sigmoid(logits)

    all_probs.append(probs.cpu())
    all_labels.append(y_batch.cpu())

  all_probs = torch.cat(all_probs).numpy()
  all_labels = torch.cat(all_labels).numpy()

  final_preds = (all_probs >= 0.14).astype(int) # 0.47 is chosen from first epoch with higher metrics in validation

  test_auc = roc_auc_score(all_labels, all_probs)
  test_precision = precision_score(all_labels, final_preds)
  test_recall = recall_score(all_labels, final_preds)
  test_f1 = f1_score(all_labels, final_preds)

  test_acc = (final_preds == all_labels).mean()

  print(f"Test Acc: {test_acc:.4f}, PR-AUC: {val_prauc:.3f},  AUROC: {test_auc:.4f}, Precision: {test_precision:.4f}, \
        Recall: {test_recall:.4f}, F1Score: {test_f1:.4f}")

Test Acc: 0.3360, PR-AUC: 0.236,  AUROC: 0.6173, Precision: 0.1712,         Recall: 0.8984, F1Score: 0.2877


In [33]:
torch.save(model.state_dict(), 'icu_deterioration.pth')

### Why accuracy is low and why recall is prioritized

The dataset is highly imbalanced(of the total only 14% positive cases), with true deterioration events being relatively rare. In such cases, accuracy becomes a misleading metric because a model can achieve high accuracy by predicting the majority (non-deterioration) class while missing critical positive cases.

This model is intentionally optimized to prioritize **recall**, ensuring that true deterioration events are detected even if it increases false positives. As a result, more patients are flagged as high risk, which lowers overall accuracy but significantly reduces the chance of missing a true positive. In an ICU setting, failing to identify a deteriorating patient is far more costly than generating an early warning that may later prove unnecessary.
